## Looping Over the Entire Leaderboard

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [ ]:
list_of_agents = ['Astra', 'Breach', 'Brimstone', 'Chamber', 'Cypher', 'Jett', 'Fade', 'KAY/O', 'Killjoy', 'Neon', 'Omen', 'Phoenix', 'Raze', 'Reyna', 'Sage', 'Skye', 'Sova', 'Viper', 'Yoru']
list_of_maps = ['Haven', 'Icebox', 'Breeze', 'Fracture', 'Ascent', 'Bind', 'Split', 'Pearl']

In [ ]:
def update_final_lists(k_d, headshot, agents, weapons, best_maps, worst_map):
    k_d_ratio_final.append(k_d)
    headshot_percent_final.append(headshot)
    top_agents_list_final.append(agents)
    top_weapons_list_final.append(weapons)
    top_three_player_maps_final.append(best_maps)
    worst_player_map_final.append(worst_map)

In [ ]:
k_d_ratio_final = []
headshot_percent_final = []
top_agents_list_final = []
top_weapons_list_final = []
top_three_player_maps_final = []
worst_player_map_final = []

In [ ]:
for page_number in range(1, 50):
    print("On page number", page_number)
    try:
        main_page_link = "https://tracker.gg/valorant/leaderboards/ranked/all/default?page="+str(page_number)+"&region=global"

        main_page = requests.get(main_page_link)

        main_page_soup = BeautifulSoup(main_page.content, 'html.parser')

        leaderboard_table = main_page_soup.find("table", {"class": "trn-table"})

        leaderboard_entries = leaderboard_table.find("tbody")

        player_list = leaderboard_entries.find_all("td", {"class": "username"})

        player_names = []
        for player in player_list:
            player_names.append(player.find("span", {"class": "trn-ign__username"}).text.strip())

        player_profile_links = []
        for player in player_list:
            a_tag = player.find("a")
            link = "https://tracker.gg" + a_tag.get('href') + "/overview?season=all"
            player_profile_links.append(link)


        rerun_iteration = True
        for i, player_profile_link in enumerate(player_profile_links):
            rerun_iteration = True
            k_d_ratio = None
            headshot_percent = None
            top_agents_list = []
            top_weapons_list = []
            top_three_player_maps = []
            worst_player_map = []
            while rerun_iteration:
                try:
                    print("Gathering data for player", player_profile_link)
                    player_profile_page = requests.get(player_profile_link)

                    player_profile_page_soup = BeautifulSoup(player_profile_page.content, 'html.parser')

                    player_headline_stats = player_profile_page_soup.find_all("div", {"class": "numbers"})
                    k_d_ratio = player_headline_stats[1].find_all("span")[1].text
                    headshot_percent = player_headline_stats[2].find_all("span")[1].text

                    top_agents = player_profile_page_soup.find_all("div", {"class": "info"})
                    for agent in top_agents:
                        if agent.text.strip() in list_of_agents:
                            top_agents_list.append(agent.text.strip())

                    top_weapons = player_profile_page_soup.find_all("div", {"class": "weapon__name"})

                    for weapon in top_weapons:
                        top_weapons_list.append(weapon.text.strip())

                    top_maps_link = player_profile_page_soup.find("a", {"class": "trn-button trn-button--block"})

                    player_maps_link = "https://tracker.gg" + top_maps_link.get('href')

                    player_maps_page = requests.get(player_maps_link)

                    player_maps_page_soup = BeautifulSoup(player_maps_page.content, 'html.parser')

                    players_maps = player_maps_page_soup.find_all('div', {"class": "info"})

                    all_maps_list = []
                    for map in players_maps:
                        if map.text.strip() in list_of_maps:
                            all_maps_list.append(map.text.strip())

                    top_three_player_maps = (all_maps_list[0:3])
                    worst_player_map = (all_maps_list[-1])
                    update_final_lists(k_d_ratio, headshot_percent, top_agents_list, top_weapons_list, top_three_player_maps, worst_player_map)
                    rerun_iteration = False
                except:
                    print("Error occurred. Retrying...")
                    k_d_ratio = None
                    headshot_percent = None
                    top_agents_list = []
                    top_weapons_list = []
                    top_three_player_maps = []
                    worst_player_map = []
                    rerun_iteration = True
    except:
        print("Web page failed to load.")

In [ ]:
k_d_ratio_final = np.array(k_d_ratio_final)
headshot_percent_final = np.array(headshot_percent_final)
top_agents_list_final = np.array(top_agents_list_final)
top_weapons_list_final = np.array(top_weapons_list_final)
top_three_player_maps_final = np.array(top_three_player_maps_final)
worst_player_map_final = np.array(worst_player_map_final)